## <center>Analiza poziomu PM2.5 w afrykańskich miastach</center>
### Zespół:
<ol>
    <li style='font-size: 20px'>Hubert Kłosowski 242424</li>
    <li style='font-size: 20px'>Krzysztof Kolanek 242425</li>
    <li style='font-size: 20px'>Kamil Małecki 242464</li>
</ol>

### Potrzebne importy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Wczytanie danych

In [ ]:
data = pd.read_csv('data\\Train.csv')
test = pd.read_csv('data\\Test.csv')

data.info()

In [ ]:
data.head()

### Rozbicie daty na składowe

In [ ]:
def change_date(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['day'] = dataframe['date'].dt.dayofweek.astype(np.int64)
    dataframe['month'] = dataframe['month'].astype(np.int64)
    return dataframe

data, test = change_date(data), change_date(test)

### Wykres przedstawiający jakość powietrza w krajach afrykańskich

In [ ]:
sns.lineplot(data=data, x='date', y='pm2_5', hue='country')
plt.title('Jakość powietrza z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 w zarejestrowanych godzinach

In [ ]:
sns.barplot(data=data, x='hour', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych godzinach z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od dnia tygodnia

In [ ]:
sns.barplot(data=data, x='day', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od miesiąca

In [ ]:
sns.barplot(data=data, x='month', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Korelacje poszczególnych grup kolumn

In [ ]:
def correlation():
    for index, column in enumerate(starts_with):
        fig, ax = plt.subplots(figsize=(10, 10))
        selected_columns = [col for col in data.columns if col.startswith(column) or col == 'pm2_5']
        sns.heatmap(data[selected_columns].corr(), annot=True, fmt='.2f', cmap='viridis', ax=ax)
        plt.tight_layout()
        plt.show()

final_ids = test['id']
data.drop(columns=['id', 'city', 'country', 'site_id', 'date', 'site_latitude', 'site_longitude', 'sulphurdioxide_so2_column_number_density_15km'], inplace=True)
test.drop(columns=['id', 'city', 'country', 'site_id', 'date', 'site_latitude', 'site_longitude', 'sulphurdioxide_so2_column_number_density_15km'], inplace=True)
starts_with = data.columns.str.split('_', expand=True).levels[0].to_frame()
starts_with.drop(['month', 'day', 'hour', 'pm2'], inplace=True)
starts_with = starts_with[0].tolist()

correlation()

## <center>Czyszczenie danych</center>

### Wykresy przedstawiające ilość NaN w danej kolumnie w zależności od przyjętej jednostki czasu

In [ ]:
def plot_nans_based_on(date_unit='day'):
    for index, column_group in enumerate(starts_with):
        similar_columns = [el for el in data.columns if el.startswith(column_group)]
        fig, ax = plt.subplots(nrows=int(np.ceil(len(similar_columns) / 4)), ncols=4, figsize=(20, 20))
        for j, column in enumerate(similar_columns):
            x_cord, y_cord = divmod(j, 4)
            nans = data[[column, date_unit]].groupby(date_unit).apply(lambda x: x.isna().sum())
            nans.plot(kind='bar', x=date_unit, y=column, ax=ax[x_cord][y_cord])
        plt.show()

plot_nans_based_on()

### 1. Uzupełnienie wartości brakujących

In [ ]:
from sklearn.impute import KNNImputer

def fill_based_on(dataframe, date_unit='day'):
    date_range = dataframe[date_unit].unique()
    for date in date_range:
        for i, column in enumerate(starts_with):
            similar_columns = [el for el in dataframe.columns if el.startswith(column)]
            df = dataframe.loc[dataframe[date_unit] == date, similar_columns]
            if not df.empty:
                if dataframe.equals(data):
                    dataframe.loc[dataframe[date_unit] == date, similar_columns] = imputers[i].fit_transform(df)
                else:
                    dataframe.loc[dataframe[date_unit] == date, similar_columns] = imputers[i].transform(df)
    return dataframe

def prepare_dataframe(dataframe):  # usuwamy kolumny o dużej liczbie wartości NaN
    to_drop = []
    for index, el in enumerate(dataframe.columns):
        if dataframe[el].isna().sum() / len(dataframe) >= 0.9:
            to_drop.append(el)
    dataframe.drop(to_drop, axis=1, inplace=True)
    return dataframe

imputers = [KNNImputer(n_neighbors=7, weights='distance') for _ in range(len(starts_with))]
data, test = prepare_dataframe(data), prepare_dataframe(test)
data, test = fill_based_on(data), fill_based_on(test)

### Wykresy pudełkowe wskazujące wartości odstające

In [ ]:
def plot_boxplots():
    for index, column_group in enumerate(starts_with):
        similar_columns = [col for col in data.columns if col.startswith(column_group)]
        rows = int(np.ceil(len(similar_columns) / 4))
        if rows >= 1:
            fig, ax = plt.subplots(nrows=int(np.ceil(len(similar_columns) / 4)), ncols=4, figsize=(20, 20))
            fig.suptitle(column_group, fontsize=15)
            for j, column in enumerate(similar_columns):
                x_cord, y_cord = divmod(j, 4)
                data[column].plot(kind='box', ax=ax[x_cord][y_cord])
            plt.show()

plot_boxplots()

### 2. Usunięcie wartości odstających

In [ ]:
from scipy.stats import zscore

def del_big_outliers(dataframe, column):
    vec, indexes = zscore(dataframe[column]), []
    for index in range(len(vec)):
        if vec[index] >= 2.5:
            indexes.append(index)
    dataframe.drop(index=indexes, inplace=True)
    return dataframe

# data = del_big_outliers(data, '')

In [ ]:
data.info()

## <center>Selekcja cech</center>

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

X, y = data.drop(['pm2_5'], axis=1), data['pm2_5']
k = 20
selector = SelectKBest(score_func=mutual_info_regression, k=k)
X = selector.fit_transform(X, y)

### Oceny cech

In [ ]:
import matplotlib.pyplot as plt

scores = dict(zip(selector.feature_names_in_, selector.scores_))
scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]
scores_df = pd.DataFrame(scores, columns=['Feature', 'Score'])

scores_df.plot(kind='bar', x='Feature', y='Score', figsize=(10, 6), rot=90, title='Oceny wybranych cech')
plt.xlabel('Cecha')
plt.ylabel('Ocena')

## <center>Transformacja danych</center>

### Potrzebne importy

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### 1. Standaryzacja danych

In [ ]:
scaler = StandardScaler()
test = test[scores_df['Feature']]
scaler.fit(X)
X, test = scaler.transform(X), scaler.transform(test.to_numpy())

### 2. Podział na zbiór testowy i treningowy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)

## <center>Część obliczeniowa</center>

### Potrzebne importy

In [ ]:
import torch
from torch import nn, optim

### 1. Wybór karty graficznej do nauki modelu

In [ ]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

X_train_tensor = torch.tensor(X_train, device=device, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, device=device, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.to_numpy(), device=device, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.to_numpy(), device=device, dtype=torch.float)
test = torch.tensor(test, device=device, dtype=torch.float)

### 2. Architektura sieci neuronowej

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(X_train_tensor.shape[1], 99),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(99, 258),
            nn.ReLU(),
            nn.Linear(258, 124),
            nn.ReLU(),
            nn.Linear(124, 64),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(64, 1),
        )
        
    def forward(self, x):
        return self.layers(x)

model = Net().to(device=device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(), 
    lr=0.01, 
    betas=(0.7, 0.999), 
    eps=1e-08, 
    weight_decay=1e-3
)

### 3. Nauka sieci neuronowej na zbiorze treningowym

In [ ]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = loss_fn(outputs.squeeze(), y_train_tensor)
    rmse = torch.sqrt(loss)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], RMSE: {rmse.item():.4f}')

### 5. Testowanie sieci neuronowej

In [ ]:
with torch.no_grad():
    y_pred = model(X_test_tensor)
    rmse = torch.sqrt(loss_fn(y_pred.squeeze(), y_test_tensor))
    print(f'RMSE: {rmse.item():.4f}')

## <center>Do wysłania</center>

In [ ]:
with torch.no_grad():
    y_pred = model(test)
    final_df = pd.concat([final_ids, pd.DataFrame.from_dict({'pm2_5': y_pred.squeeze().numpy(force=True)})], axis=1)
    final_df.rename(columns={'id': 'ID'}, inplace=True)
    final_df['pm2_5'] = final_df['pm2_5'].apply(lambda x: np.sqrt(x))
    final_df.to_csv('result.csv', index=False)

### Dodatkowe informacje
<ol>
    <li>The 15km SO2 band is ingested only when solar_zenith_angle < 70.</li>
    <li>Because of noise on the data, negative vertical column values are often observed in particular over clean regions or for low SO2 emissions. It is recommended not to filter these values except for outliers, i.e. for vertical columns lower than -0.001 mol/m^2.</li>
</ol>